# Descargar el ultimo mes disponible en el Tableau FONASA


In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install TableauScraper

import time
import calendar
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from tableauscraper import TableauScraper as TS
from datetime import datetime, timedelta
from google.colab import files

# Funciones predefinidas
def get_filter_dict(filter_id, dashboard_url):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)
    try:
        driver.get(dashboard_url)
        time.sleep(2)
        iframe = driver.find_element(By.TAG_NAME, 'iframe')
        driver.switch_to.frame(iframe)
        element_to_click = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, filter_id)))
        element_to_click.click()
        menu = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, f"{filter_id}_menu")))
        fitext_elements = menu.find_elements(By.CLASS_NAME, "FIText")
        fitext_texts = [element.get_attribute("title") for element in fitext_elements]
        filter_dict = {filter_name: index for index, filter_name in enumerate(fitext_texts[1:], start=0)}
        return filter_dict
    finally:
        close_filter_with_esc(driver)
        driver.quit()

def initialize_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)
    return driver

def open_dashboard(driver, dashboard_url):
    driver.get(dashboard_url)
    time.sleep(2)
    iframe = driver.find_element(By.TAG_NAME, 'iframe')
    driver.switch_to.frame(iframe)

def get_link_value(driver):
    link_input = driver.find_element(By.CSS_SELECTOR, '.tab-shareLinkSection input.tab-shareInput')
    return link_input.get_attribute("value")

def click_element(element):
    element.click()
    time.sleep(2)

def select_checkbox(checkbox_element):
    if not checkbox_element.is_selected():
        driver.execute_script("arguments[0].click();", checkbox_element)
    time.sleep(2)

def close_filter_with_esc(driver):
    actions = ActionChains(driver)
    actions.send_keys(Keys.ESCAPE)
    actions.perform()
    time.sleep(2)

# Varibales
dashboard_url = 'https://public.tableau.com/shared/CJPX54XFH?:display_count=n&:origin=viz_share_link'
final_df = pd.DataFrame(columns=[
    "Establecimiento Origen-alias",
    "Problema de salud-alias",
    "Región de Origen-alias",
    "Principio Activo",
    "Mes",
    "Año",
    "Estado",
    "SUM(Number of Records)-alias"
])

# Obtenemos los diccionarios relevantes
estado_dict = get_filter_dict('tableau_base_widget_LegacyCategoricalQuickFilter_3', dashboard_url)
pa_dict = get_filter_dict('tableau_base_widget_LegacyCategoricalQuickFilter_4', dashboard_url)
mes_dict = get_filter_dict('tableau_base_widget_LegacyCategoricalQuickFilter_6', dashboard_url)
last_12_indices = list(mes_dict.values())[-12:]
mes_indices_and_values = [(mes_text, mes_index) for mes_text, mes_index in mes_dict.items()]

# Iterar a través de los índices en estado_dict y pa_dict
for estado_text, estado_index in estado_dict.items():
    for pa_text, pa_index in pa_dict.items():
        for mes_text, mes_index in mes_indices_and_values:
            if mes_index in last_12_indices:
                while True:
                    try:
                        # Abrir el navegador y cargar la URL del dashboard
                        driver = initialize_driver()
                        open_dashboard(driver, dashboard_url)

                        # Seleccionar el mes respectivo en el tablero de Fonasa
                        element_to_click = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'tableau_base_widget_LegacyCategoricalQuickFilter_6')))
                        click_element(element_to_click)
                        checkbox_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, f'FI_federated.0r9zjyb0o089w11cehq2v1tugtmx,my:Fecha Confirmación &#47; Descartes:ok6074625098531203344_6459929784170675544_{mes_index}')))
                        select_checkbox(checkbox_element)
                        close_filter_with_esc(driver)

                        # Seleccionar el estado respectivo
                        element_to_click = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'tableau_base_widget_LegacyCategoricalQuickFilter_3')))
                        click_element(element_to_click)
                        checkbox_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, f'FI_federated.0r9zjyb0o089w11cehq2v1tugtmx,none:Estado:nk6074625098531203344_6459929784170675544_{estado_index}')))
                        select_checkbox(checkbox_element)
                        close_filter_with_esc(driver)

                        # Seleccionar el principio activo respectivo
                        element_to_click = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'tableau_base_widget_LegacyCategoricalQuickFilter_4')))
                        click_element(element_to_click)
                        checkbox_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, f'FI_federated.0r9zjyb0o089w11cehq2v1tugtmx,none:Principio activo:nk6074625098531203344_6459929784170675544_{pa_index}')))
                        select_checkbox(checkbox_element)
                        close_filter_with_esc(driver)

                        # Obtener el enlace para compartir el tablero generado
                        compartir_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'share-ToolbarButton')))
                        click_element(compartir_element)
                        share_dialog = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-tb-test-id="share-Dialog-Floater"]')))
                        link_value = get_link_value(driver)
                        close_filter_with_esc(driver)

                        # Procesar la URL para generar el DataFrame y concatenar al DataFrame final
                        Month, Year = mes_text.split(" ")
                        Month = Month.capitalize()
                        ts = TS()
                        ts.loads(link_value)
                        ws = ts.getWorksheet("0501 CD Establecimiento")
                        if not ws.data.empty:
                            data = ws.data
                            df = pd.DataFrame(data, columns=final_df.columns)
                            df["Principio Activo"] = pa_text
                            df["Estado"] = estado_text
                            df["Mes"] = Month
                            df["Año"] = Year
                            df = df[(df["Establecimiento Origen-alias"] != "%all%") & (df["Problema de salud-alias"] != "%all%")]
                            final_df = pd.concat([final_df, df], ignore_index=True)

                        # Cerrar el navegador
                        driver.quit()
                        break

                    except Exception as e:
                        print(f"Error occurred: {e}")
                        print("Refreshing the page and retrying the last element...")
                        driver.quit()
                        continue

output_filename = f"FONASA_{Month}_{Year}.csv"
final_df.to_csv(output_filename, index=False, encoding="latin-1")
files.download(output_filename)
print(final_df)

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [993 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [966 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [498 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archiv

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

                           Establecimiento Origen-alias  \
0     Hospital Dr. Félix Bulnes Cerda (Santiago, Qui...   
1                          Hospital San José (Victoria)   
2                 Hospital San Juan de Dios (La Serena)   
3                 Hospital San Juan de Dios (Los Andes)   
4                                Clínica Ciudad del Mar   
...                                                 ...   
5301  Hospital Barros Luco Trudeau (Santiago, San Mi...   
5302                                     Clínica Dávila   
5303     Hospital Dr. Hernán Henríquez Aravena (Temuco)   
5304               Hospital Clínico Regional (Valdivia)   
5305  Hospital Dr. Félix Bulnes Cerda (Santiago, Qui...   

              Problema de salud-alias Región de Origen-alias Principio Activo  \
0     Artritis Reumatoide Refractaria          Metropolitana        Abatacept   
1     Artritis Reumatoide Refractaria           La Araucanía        Abatacept   
2     Artritis Reumatoide Refractaria           